In [1]:
import sys
import os
from IPython import get_ipython

sys.path.append(os.path.abspath("./lib"))
if 'autoreload' not in get_ipython().magics_manager.magics['line']:
    %load_ext autoreload
%autoreload 2

from lib import dataloading as dl
from lib import tokenizer as tk
import tokenizers
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
import sage_tokenizer

ModuleNotFoundError: No module named 'sage_tokenizer'

In [19]:
data_df = dl.load_conllu(
    r"D:\Dropbox\Bachlorarbeit\Datasets\Universal Dependencies 2.15\ud-treebanks-v2.15\UD_English-GUM\en_gum-ud-train.conllu"
)
data_df = dl.clear_non_UPOS_tags(data_df)
print(data_df.head())

Dropped 2810 rows with non-UPOS tags 
Tags dropped: ['_']
            FORM         LEMMA   UPOS XPOS        FEATS HEAD DEPREL  \
ID                                                                    
1      Aesthetic     aesthetic    ADJ   JJ   Degree=Pos    2   amod   
2   Appreciation  appreciation   NOUN   NN  Number=Sing    0   root   
3            and           and  CCONJ   CC            _    5     cc   
4        Spanish       Spanish    ADJ   JJ   Degree=Pos    5   amod   
5            Art           art   NOUN   NN  Number=Sing    2   conj   

          DEPS                                               MISC  
ID                                                                 
1       2:amod  Discourse=organization-heading:1->57:8:grf-ly-...  
2       0:root                       Entity=1)|MSeg=Appreciat-ion  
3         5:cc                                                  _  
4       5:amod     Entity=(2-abstract-new-cf2-2-sgl|MSeg=Span-ish  
5   2:conj:and                      

In [20]:
vocab_size = 1000
upos_tags = [
        "ADJ",
        "ADP",
        "ADV",
        "AUX",
        "CCONJ",
        "DET",
        "INTJ",
        "NOUN",
        "NUM",
        "PART",
        "PRON",
        "PROPN",
        "PUNCT",
        "SCONJ",
        "SYM",
        "VERB",
        "X",
    ]
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]

In [21]:
data_df["UPOS"].value_counts(normalize=True).sort_index()

UPOS
ADJ      0.065591
ADP      0.094304
ADV      0.047767
AUX      0.053751
CCONJ    0.032844
DET      0.081215
INTJ     0.009768
NOUN     0.166281
NUM      0.019317
PART     0.023972
PRON     0.084079
PROPN    0.058059
PUNCT    0.138616
SCONJ    0.016136
SYM      0.001627
VERB     0.104742
X        0.001932
Name: proportion, dtype: float64

In [22]:
for algorithm in ["BPE", "WordPiece", "Unigram"]:
    path = Path(r"D:\Dropbox\Bachlorarbeit\Tokenization by Word Tag\Tokenization by Word Tag Tokenizers") / f"upos_{algorithm.lower()}_tokenizer.json"
    tokenizer = tk.train_and_merge_tokenizers(
        data_df,
        tokenizer_algorithm=algorithm,
        vocab_size=1000,
        allocation="proportional",
        save_path=str(path)
    )

In [23]:
vocab_size = 1000
tokenizer = tk.train_tokenizer(data_df["FORM"].values.tolist(), vocab_size, algorithm="bpe")
tokenizer.save(r"D:\Dropbox\Bachlorarbeit\Tokenization by Word Tag\Tokenization by Word Tag Tokenizers\classic_bpe_tokenizer.json")

tokenizer = tk.train_tokenizer(data_df["FORM"].values.tolist(), vocab_size, algorithm="wordpiece")
tokenizer.save(r"D:\Dropbox\Bachlorarbeit\Tokenization by Word Tag\Tokenization by Word Tag Tokenizers\classic_wordpiece_tokenizer.json")

tokenizer = tk.train_tokenizer(data_df["FORM"].values.tolist(), vocab_size, algorithm="unigram")
tokenizer.save(r"D:\Dropbox\Bachlorarbeit\Tokenization by Word Tag\Tokenization by Word Tag Tokenizers\classic_unigram_tokenizer.json")